In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [3]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [5]:
gan = GAN()
gan.train(epochs=3000, batch_size=32, sample_interval=200)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

C:\Users\Tejo Vinay\Anaconda3\lib\site-packages\keras\engine\training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.620411, acc.: 68.75%] [G loss: 0.674147]


C:\Users\Tejo Vinay\Anaconda3\lib\site-packages\keras\engine\training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.464737, acc.: 65.62%] [G loss: 0.694008]
2 [D loss: 0.316827, acc.: 84.38%] [G loss: 0.859018]
3 [D loss: 0.284958, acc.: 85.94%] [G loss: 1.010342]
4 [D loss: 0.222347, acc.: 98.44%] [G loss: 1.177935]
5 [D loss: 0.207029, acc.: 96.88%] [G loss: 1.317182]
6 [D loss: 0.182027, acc.: 100.00%] [G loss: 1.478160]
7 [D loss: 0.149242, acc.: 100.00%] [G loss: 1.608848]
8 [D loss: 0.128671, acc.: 100.00%] [G loss: 1.727287]
9 [D loss: 0.138154, acc.: 100.00%] [G loss: 1.837904]
10 [D loss: 0.108443, acc.: 100.00%] [G loss: 1.843769]
11 [D loss: 0.104608, acc.: 100.00%] [G loss: 1.989546]
12 [D loss: 0.086270, acc.: 100.00%] [G loss: 2.013896]
13 [D loss: 0.086288, acc.: 100.00%] [G loss: 2.138079]
14 [D loss: 0.078538, acc.: 100.00%] [G loss: 2.169658]
15 [D loss: 0.065812, acc.: 100.00%] [G loss: 2.323133]
16 [D loss: 0.069235, acc.: 100.00%] [G loss: 2.352100]
17 [D loss: 0.065495, acc.: 100.00%] [G loss: 2.432600]
18 [D loss: 0.056391, acc.: 100.00%] [G loss: 2.448395]
19 [D 

...

2341 [D loss: 0.549892, acc.: 73.44%] [G loss: 0.996311]
2342 [D loss: 0.573033, acc.: 65.62%] [G loss: 0.985439]
2343 [D loss: 0.601187, acc.: 68.75%] [G loss: 0.956972]
2344 [D loss: 0.604175, acc.: 65.62%] [G loss: 0.951755]
2345 [D loss: 0.638586, acc.: 65.62%] [G loss: 0.981440]
2346 [D loss: 0.604540, acc.: 70.31%] [G loss: 0.935892]
2347 [D loss: 0.547274, acc.: 76.56%] [G loss: 1.040666]
2348 [D loss: 0.569710, acc.: 73.44%] [G loss: 0.949306]
2349 [D loss: 0.604093, acc.: 67.19%] [G loss: 0.961648]
2350 [D loss: 0.640492, acc.: 62.50%] [G loss: 0.889974]
2351 [D loss: 0.627032, acc.: 56.25%] [G loss: 1.011252]
2352 [D loss: 0.559418, acc.: 82.81%] [G loss: 1.059956]
2353 [D loss: 0.598500, acc.: 70.31%] [G loss: 1.081516]
2354 [D loss: 0.604259, acc.: 67.19%] [G loss: 1.097576]
2355 [D loss: 0.601967, acc.: 67.19%] [G loss: 0.968793]
2356 [D loss: 0.582080, acc.: 68.75%] [G loss: 1.001073]
2357 [D loss: 0.590488, acc.: 73.44%] [G loss: 1.024175]
2358 [D loss: 0.552324, acc.: 7

2484 [D loss: 0.557669, acc.: 73.44%] [G loss: 0.922273]
2485 [D loss: 0.531857, acc.: 78.12%] [G loss: 0.960535]
2486 [D loss: 0.526139, acc.: 81.25%] [G loss: 0.966524]
2487 [D loss: 0.536221, acc.: 81.25%] [G loss: 0.960721]
2488 [D loss: 0.617602, acc.: 68.75%] [G loss: 0.936649]
2489 [D loss: 0.672121, acc.: 60.94%] [G loss: 0.957550]
2490 [D loss: 0.568975, acc.: 76.56%] [G loss: 0.962043]
2491 [D loss: 0.597116, acc.: 70.31%] [G loss: 0.978723]
2492 [D loss: 0.606563, acc.: 62.50%] [G loss: 1.018820]
2493 [D loss: 0.551123, acc.: 75.00%] [G loss: 1.030090]
2494 [D loss: 0.647627, acc.: 65.62%] [G loss: 0.968987]
2495 [D loss: 0.636331, acc.: 57.81%] [G loss: 0.964735]
2496 [D loss: 0.567637, acc.: 75.00%] [G loss: 0.941296]
2497 [D loss: 0.576494, acc.: 71.88%] [G loss: 0.968262]
2498 [D loss: 0.628672, acc.: 65.62%] [G loss: 0.892757]
2499 [D loss: 0.631274, acc.: 65.62%] [G loss: 1.037317]
2500 [D loss: 0.601732, acc.: 67.19%] [G loss: 0.992042]
2501 [D loss: 0.592038, acc.: 7

2628 [D loss: 0.643245, acc.: 60.94%] [G loss: 0.958930]
2629 [D loss: 0.640754, acc.: 64.06%] [G loss: 1.076843]
2630 [D loss: 0.585471, acc.: 64.06%] [G loss: 1.024471]
2631 [D loss: 0.649164, acc.: 60.94%] [G loss: 0.956094]
2632 [D loss: 0.588773, acc.: 65.62%] [G loss: 1.007280]
2633 [D loss: 0.684072, acc.: 60.94%] [G loss: 0.889637]
2634 [D loss: 0.576282, acc.: 71.88%] [G loss: 0.902100]
2635 [D loss: 0.640758, acc.: 64.06%] [G loss: 0.926581]
2636 [D loss: 0.627111, acc.: 64.06%] [G loss: 1.076671]
2637 [D loss: 0.606625, acc.: 71.88%] [G loss: 0.958867]
2638 [D loss: 0.597558, acc.: 70.31%] [G loss: 0.927415]
2639 [D loss: 0.654573, acc.: 67.19%] [G loss: 0.990095]
2640 [D loss: 0.594561, acc.: 70.31%] [G loss: 0.899515]
2641 [D loss: 0.615041, acc.: 62.50%] [G loss: 0.922799]
2642 [D loss: 0.565597, acc.: 73.44%] [G loss: 0.996944]
2643 [D loss: 0.602629, acc.: 70.31%] [G loss: 0.889999]
2644 [D loss: 0.553356, acc.: 73.44%] [G loss: 0.989248]
2645 [D loss: 0.585631, acc.: 6

2775 [D loss: 0.547454, acc.: 78.12%] [G loss: 0.974239]
2776 [D loss: 0.637918, acc.: 64.06%] [G loss: 1.066369]
2777 [D loss: 0.549554, acc.: 71.88%] [G loss: 0.952675]
2778 [D loss: 0.616702, acc.: 68.75%] [G loss: 1.023439]
2779 [D loss: 0.588674, acc.: 70.31%] [G loss: 0.907113]
2780 [D loss: 0.641072, acc.: 64.06%] [G loss: 0.960248]
2781 [D loss: 0.566026, acc.: 65.62%] [G loss: 0.964340]
2782 [D loss: 0.610189, acc.: 67.19%] [G loss: 0.975936]
2783 [D loss: 0.529276, acc.: 73.44%] [G loss: 1.032432]
2784 [D loss: 0.590657, acc.: 67.19%] [G loss: 0.974118]
2785 [D loss: 0.609074, acc.: 64.06%] [G loss: 0.948745]
2786 [D loss: 0.648903, acc.: 64.06%] [G loss: 0.858416]
2787 [D loss: 0.585748, acc.: 68.75%] [G loss: 0.926284]
2788 [D loss: 0.479482, acc.: 79.69%] [G loss: 1.002083]
2789 [D loss: 0.582917, acc.: 68.75%] [G loss: 0.933846]
2790 [D loss: 0.616804, acc.: 65.62%] [G loss: 0.895655]
2791 [D loss: 0.621150, acc.: 65.62%] [G loss: 0.994352]
2792 [D loss: 0.638525, acc.: 6

2918 [D loss: 0.635717, acc.: 62.50%] [G loss: 0.912909]
2919 [D loss: 0.613974, acc.: 64.06%] [G loss: 0.937081]
2920 [D loss: 0.558916, acc.: 65.62%] [G loss: 0.926949]
2921 [D loss: 0.536335, acc.: 71.88%] [G loss: 0.903874]
2922 [D loss: 0.576111, acc.: 70.31%] [G loss: 0.938112]
2923 [D loss: 0.556588, acc.: 75.00%] [G loss: 0.934744]
2924 [D loss: 0.593078, acc.: 62.50%] [G loss: 1.014078]
2925 [D loss: 0.548234, acc.: 76.56%] [G loss: 1.046566]
2926 [D loss: 0.616588, acc.: 71.88%] [G loss: 1.080868]
2927 [D loss: 0.616373, acc.: 64.06%] [G loss: 1.034196]
2928 [D loss: 0.578613, acc.: 75.00%] [G loss: 1.058354]
2929 [D loss: 0.625932, acc.: 67.19%] [G loss: 0.980990]
2930 [D loss: 0.664459, acc.: 56.25%] [G loss: 0.914487]
2931 [D loss: 0.596833, acc.: 67.19%] [G loss: 0.965099]
2932 [D loss: 0.575271, acc.: 67.19%] [G loss: 1.001899]
2933 [D loss: 0.611162, acc.: 60.94%] [G loss: 0.978548]
2934 [D loss: 0.590801, acc.: 64.06%] [G loss: 0.913100]
2935 [D loss: 0.606510, acc.: 7